Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [233]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], db_config['pwd'], db_config['host'], db_config['port'],
db_config['db']) 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [234]:
query1 = '''select * from books'''
books = pd.io.sql.read_sql(query1, con = engine)

In [235]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [236]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [5]:
query2 = '''select * from authors'''
authors = pd.io.sql.read_sql(query2, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [7]:
query3 = '''select * from publishers'''
publishers = pd.io.sql.read_sql(query3, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [9]:
query4 = '''select * from ratings'''
ratings = pd.io.sql.read_sql(query4, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [11]:
query5 = '''select * from reviews'''
reviews = pd.io.sql.read_sql(query5, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


### Для каждой книги посчитайте количество обзоров и среднюю оценку:

In [205]:
query = '''select distinct books.title, count(distinct review_id) as review_cnt, avg(rating) as mean_rt 
           from books inner join reviews on reviews.book_id = books.book_id
           inner join ratings on ratings.book_id = books.book_id
           group by books.title order by review_cnt DESC'''
sql1 = pd.io.sql.read_sql(query, con = engine)

In [206]:
pd.options.display.max_colwidth = 120
sql1.head(20)

,title,review_cnt,mean_rt
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Eat Pray Love,6,3.395833
3,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.287500
4,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.414634
5,Outlander (Outlander #1),6,4.125000
6,The Alchemist,6,3.789474
7,The Book Thief,6,4.264151
8,The Catcher in the Rye,6,3.825581
9,The Curious Incident of the Dog in the Night-Time,6,4.081081


Больше всего обзоров на книгу "Мемуары Гейши" - 8. В топ-20 книг по количеству рецензий средние оценки по пятибалльной шкале довольно высокие - больше половины

### Посчитайте, сколько книг вышло после 1 января 2000 года:

In [15]:
query = '''select count(distinct book_id) as cnt_book from books 
           where cast(publication_date as date) >= '2000-01-02' '''
sql2 = pd.io.sql.read_sql(query, con = engine)

In [16]:
sql2

,cnt_book
0,819


819 книг выпущено после 1 января 2000 года. С одной стороны это не очень много, а с другой - все-так в базе данных в основном книги-хитыб бестселлеры вроде "Гарри Поттера", "Ешь, Молись, Люби", "Сумерки"

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры:

In [39]:
query = '''select distinct publishers.publisher,  count(distinct books.book_id) as cnt_book 
           from  publishers
           inner join books on books.publisher_id = publishers.publisher_id
           where books.num_pages > 50
           group by publishers.publisher
           order by cnt_book desc'''

In [40]:
sql3 = pd.io.sql.read_sql(query, con = engine)
sql3

,publisher,cnt_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Wordsworth Editions,1
330,Workman Publishing Company,1
331,Wyatt Book,1
332,Yale University Press,1


Больше всего книг выпустило издательство Penguin Books

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками:

In [227]:
query = '''select t.author as max_author, max(t.avg_rate) as max_rate
           from (
           select distinct authors.author, books.book_id as book,
           avg(ratings.rating) as avg_rate, count(ratings.rating) as cnt_rate
           from authors 
           inner join books on books.author_id = authors.author_id
           inner join ratings on ratings.book_id = books.book_id
           group by authors.author, book
           having count(ratings.book_id) > 50) as t
           group by max_author
           order by max_rate desc
           '''

In [228]:
sql5 = pd.io.sql.read_sql(query, con = engine)
sql5

,max_author,max_rate
0,J.K. Rowling/Mary GrandPré,4.414634
1,J.R.R. Tolkien,4.391892
2,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,Dan Brown,3.830508
7,J.D. Salinger,3.825581
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
9,William Shakespeare/Paul Werstine/Barbara A. Mowat,3.787879


Самая высокая средняя оценка у Джоан Роулинг (Мэри Гранпрэ иллстрировала серию "Гарри Поттер")

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [231]:
query = '''select round(avg(t.cnt_review)) as avg_review
           from 
           (
           select reviews.username, count(distinct reviews.review_id) as cnt_review 
           from reviews
           inner join ratings on ratings.username = reviews.username
           group by reviews.username
           having count(distinct ratings.rating_id) > 50) as t
           '''

In [232]:
sql4 = pd.io.sql.read_sql(query, con = engine)
sql4

,avg_review
0,24.0


Очевидно, что пользователи, ставящие более 50 оценок увлеченные книголюбы, которые еще и пишут достоточно рецензий на книги

In [ ]:
y_true = []